In [1]:
import pandas as pd
import os
from pathlib import Path

from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
class TrainModel:
    """
    Clase encargada de entrenar y evaluar modelos de ml
    """
    def __init__(self,train_file_path,test_file_path=None):
        """
        Función que inicializa la clase con la ruta el archivo y verifica
        que exista.

        Attributes
        ----------
        file_path: str
            Ruta del archivo csv

        Raise
        FileNotFoundError
            Si el archivo no existe.
        """
        self.train_file_path = Path(train_file_path)
        self.test_file_path = Path(test_file_path) if test_file_path else None
        if not self.train_file_path.exists():
            raise FileNotFoundError(f"No existe la ruta {self.train_file_path}")

        if self.test_file_path and not self.test_file_path.exists():
            raise FileNotFoundError(f"No existe la ruta {self.test_file_path}")
        
        self.data = None
        self.test_data =None
        self.pipeline = None
    
    def load_dataset(self):
        """
        Lee el archivo csv de entrenamiento indicado en el self.train_file_path y lo carga como un
        dataframe de pandas.

        return
        ------
        pd.DataFrame
            DataFrame que contiene los datos leidos del archivo csv.

        Raises
        ------
        pandas.errors.EmptyDataError
            Si el archivo está vacío

        pandas.errors.ParserError
            Si el archivo CSV tiene errores de formato o no puede ser parseado correctamente.
        """

        self.data = pd.read_csv(self.train_file_path)
        return self.data
    
    def load_test_dataset(self):

        """
        Carga el dataset de test (si se proporcionó).

        return
        ------
        pd.DataFrame
            DataFrame que contiene los datos leidos del archivo csv.

        Raises
        ------
        ValueError
            Si no se proporciona la ruta del dataset de prueba
        
        pandas.errors.EmptyDataError
            Si el archivo está vacío

        pandas.errors.ParserError
            Si el archivo CSV tiene errores de formato o no puede ser parseado correctamente.
        """

        if self.test_file_path is None:
            raise ValueError("No se proporcionó la ruta del dataset de test.")
        self.test_data = pd.read_csv(self.test_file_path)

        return self.test_data
    
    def train_pipeline(self, target ='default_12m',parameters = None):
        """
        Entrena un pipeline con RobustScaler y DecisionTreeClassifier

        Attributes
        ----------
        target: str
            Nombre de la columna objetivo, por decfecto es 'defaul_12m'

        parameters: dict
            Diccionario con los mejores parámetros para entrenar el modelo

        Raise
        -----
        ValueError
            Si la variable target no existe en el dataset

        Return
        ------
        sklearn.pipeline
            pipeline de skalearn entrenada

        """
        if target not in self.data.columns:
            raise ValueError(f"La variable objetivo '{target}' no se encuentra en los datos.")
        
        independientes = self.data.drop(columns=[target],axis=1)
        objetivo = self.data[target]

        parameters = parameters or {}
        pipeline = Pipeline([ 
            ('scaler',RobustScaler()), 
            ('classifier',DecisionTreeClassifier(**parameters)) ])

        pipeline.fit(independientes, objetivo)
        self.pipeline = pipeline
    
        return self.pipeline
    
    def test_pipeline(self,target = 'default_12m',pos_label=1,print_metrics=True):
        """
        Se encarga de evaluar el modelo entrenado

        Attributes
        ----------
        target: str
            Nombre de la columna objetivo, por decfecto es 'defaul_12m'

        pos_label: int
            Clase que se considera positiva, por defecto es 1

        print_metrics: Booleano
            Indica si desea imprimir métricas de rendimiento
        
        Raises
        ------
        ValueError
            Si los datos de test no están cargados, si el pipeline no está entrenado y si 
            la variable objetivo no está en los datos de test.

        Return
        ------
        dict
            Diccionario con las métricas de evaluación del modelo

        """
        if self.pipeline is None:
            raise ValueError("El pipeline no está entrenado. Ejecuta primero 'train_pipeline()'.")
        if self.test_data is None:
            raise ValueError("Los datos de test no están cargados. Ejecuta 'load_test_dataset()'.")
        if target not in self.test_data.columns:
            raise ValueError(f"La variable objetivo '{target}' no se encuentra en los datos de test.")
        
        X_test = self.test_data.drop(columns=[target])
        y_test = self.test_data[target]
        
        y_pred = self.pipeline.predict(X_test)
        f1 = f1_score(y_test, y_pred, pos_label=pos_label)
        precision = precision_score(y_test, y_pred, pos_label=pos_label)
        recall = recall_score(y_test, y_pred, pos_label=pos_label)

        metrics = {
            'f1': f1,
            'precision': precision,
            'recall': recall
            }
        
        if print_metrics:
            print("=== MÉTRICAS DEL MODELO ===")
            print(f"F1-Score: {f1:.4f}")
            print(f"Precisión: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
        return metrics


In [ ]:
import mlflow
from mlflow.client import MlflowClient

class RegisterModel:
    """
    Configura un cliente para interacturar con MLflow. El propósito general
    es inicializar un objeto para gestionar experimentos y modelos de MLflow.
    """
    def __init__(self, name_model: str, tracking_uri: str = None):
        """
        """
        self.name_model = name_model
        self.tracking_uri = tracking_uri or os.getenv("MLFLOW_TRACKING_URI", 
                                                      "http://localhost:5000")
        # Configurar la URI global de MLflow
        mlflow.set_tracking_uri(self.tracking_uri)
        self.client = MlflowClient(tracking_uri=self.tracking_uri)


In [ ]:
import mlflow
from mlflow.client import MlflowClient

# Configuración de MLflow
tracking_uri = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient(tracking_uri=tracking_uri)

    with mlflow.start_run(run_name=f"store-{store_id}"):
        # Loggear parámetros
        mlflow.log_params({
            'store_id': store_id,
            'train_fraction': train_fraction,
            'yearly_seasonality': seasonality['yearly'],
            'weekly_seasonality': seasonality['weekly'],
            'daily_seasonality': seasonality['daily'],
            'interval_width': 0.95
        })
        
        # Loggear métricas
        mlflow.log_metric("training_samples", len(df_train))
        mlflow.log_metric("test_samples", len(df_test))
        
        # Registrar el modelo - FORMA CORRECTA para MLflow >= 2.0
        mlflow.prophet.log_model(model, artifact_path="model")
        
        # Registrar el nombre del modelo manualmente
        mlflow.set_tag("model_name", model_name)
        mlflow.set_tag("store_id", store_id)
        
        print(f"✅ Modelo {model_name} registrado en MLflow")
class ModelRegistrar:
    """
    """

    def __init__(self,model_name):
        

Probando el código

In [3]:
project_root = next(p for p in Path.cwd().parents if (p / 'data').exists())
file_path = lambda file: os.path.join(project_root,'data/processed',file)

train_model = TrainModel(
    train_file_path=file_path('covalto_sme_credit_train.csv'),
    test_file_path=file_path('covalto_sme_credit_test.csv')
)
train_model.load_dataset()
train_model.load_test_dataset()

modelo = train_model.train_pipeline(
    target = 'default_12m',
    parameters = {
        'criterion': 'gini', 
        'max_depth': 2, 
        'min_samples_split': 8, 
        'min_samples_leaf': 19, 
        'max_features': None, 
        'class_weight': None}
)

metrics = train_model.test_pipeline()

=== MÉTRICAS DEL MODELO ===
F1-Score: 0.8000
Precisión: 0.8000
Recall: 0.8000
